## Summary

---

## Imports

In [1]:
import os
import tempfile
import urllib.request
from datetime import datetime
from pathlib import Path

import dotenv
import elaspic2 as el2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns
import torch
from kmbio import PDB
from kmtools import structure_tools
from tqdm.auto import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("35_cagi6_hmbs_el2").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/home/kimlab5/strokach/workspace/elaspic/elaspic2-cagi6/notebooks/31_cagi6_hmbs_el2')

In [3]:
UNIPROT_ID = "P08397"

UNIPROT_ID

'P08397'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cpu')

In [5]:
version = datetime.now().isoformat(timespec="hours")

version

'2021-09-11T17'

## Load data

In [12]:
def load_sequence(sequence_file):
    with sequence_file.open("rt") as fin:
        data = fin.read()
    chunks = []
    for line in data.split("\n"):
        if line.startswith(">"):
            continue
        chunks.append(line.strip())
    return "".join(chunks)

In [13]:
sequence_file = NOTEBOOK_DIR.parent.joinpath("30_cagi6_hmbs", f"{UNIPROT_ID}.fasta")

assert sequence_file.is_file()

In [14]:
sequence = load_sequence(sequence_file)

sequence[:5]

'MSGNG'

In [15]:
structure_file = NOTEBOOK_DIR.parent.joinpath("30_cagi6_hmbs", f"{UNIPROT_ID}.pdb")

assert structure_file.is_file()

In [16]:
results_to_fill_file = NOTEBOOK_DIR.parent.joinpath("30_cagi6_hmbs", "results-to-fill.parquet")

results_to_fill_df = pq.read_table(results_to_fill_file).to_pandas()

In [17]:
results_to_fill_df.head(2)

,aa_substitution,score,sd,comments,mut
0,p.Ala112Arg,*,*,*,A112R
1,p.Ala112Asn,*,*,*,A112N


## Run `ELASPIC2`

### Initialize the `ELASPIC2` model



In [20]:
model = el2.ELASPIC2(device=device)

Some weights of the model checkpoint at /home/kimlab5/strokach/workspace/elaspic/elaspic2/src/elaspic2/plugins/protbert/data/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Make predictions

In [20]:
protein_features = model.build(
    structure_file=structure_file,
    protein_sequence=sequence,
    ligand_sequence=None,
    remove_hetatms=True,
)

protein_features

ELASPIC2Data(is_interface=False, protbert_data=ProtBertData(sequence='MSGNGNAAATAEENSPKMRVIRVGTRKSQLARIQTDSVVATLKASYPGLQFEIIAMSTTGDKILDTALSKIGEKSLFTKELEHALEKNEVDLVVHSLKDLPTVLPPGFTIGAICKRENPHDAVVFHPKFVGKTLETLPEKSVVGTSSLRRAAQLQRKFPHLEFRSIRGNLNTRLRKLDEQQEFSAIILATAGLQRMGWHNRVGQILHPEECMYAVGQGALGVEVRAKDQDILDLVGVLHDPETLLRCIAERAFLRHLEGGCSVPVAVHTAMKDGQLYLTGGVWSLDGSDSIQETMQATIHVPAQHEDGPEDDPQLVGITARNIPRGPQLAAQNLGISLANLLLSKGAKNILDVARQLNDAH'), proteinsolver_data=Data(edge_attr=[19086, 2], edge_index=[2, 19086], x=[361]))

In [30]:
mutation_features = list(
    tqdm(
        (model.analyze_mutation(mut, protein_features) for mut in result_df["mut"]),
        total=len(result_df),
    )
)

In [33]:
# In all cases, higher scores means less stable (same as ΔΔG)
result_df["protbert_score"] = [
    f["protbert_core_score_wt"] - f["protbert_core_score_mut"] for f in mutation_features
]
result_df["proteinsolver_score"] = [
    f["proteinsolver_core_score_wt"] - f["proteinsolver_core_score_mut"] for f in mutation_features
]
result_df["el2_score"] = model.predict_mutation_effect(mutation_features).tolist()

In [19]:
output_file = NOTEBOOK_DIR.joinpath("results-el2.parquet")

pq.write_table(
    pa.Table.from_pandas(result_df, preserve_index=False),
)

True